<a href="https://colab.research.google.com/github/ayulockin/TF-MSN/blob/main/notebooks/TF_Exponential_Moving_Average_Weights_Sharing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

import numpy as np

In [4]:
x = np.random.randint(255, size=(100, 28, 28, 1)).astype(np.float32)
y = np.random.randint(10, size=(100,)).astype(np.float32)

x.shape, y.shape

((100, 28, 28, 1), (100,))

In [18]:
def build_anchor_model():
    inputs = layers.Input(shape=(28, 28, 1))
    x = layers.Conv2D(3, 3, activation="relu")(inputs)
    x = layers.GlobalAvgPool2D()(x)
    classifier = layers.Dense(10, activation="softmax")(x)

    return models.Model(inputs, classifier, name="anchor_model")

def build_target_model():
    inputs = layers.Input(shape=(28, 28, 1))
    x = layers.Conv2D(3, 3, activation="relu")(inputs)
    x = layers.GlobalAvgPool2D()(x)
    classifier = layers.Dense(10, activation="softmax")(x)

    return models.Model(inputs, classifier, name="target_model")

In [27]:
tf.keras.backend.clear_session()
anchor_model = build_anchor_model()
anchor_model.summary()

Model: "anchor_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 3)         30        
                                                                 
 global_average_pooling2d (G  (None, 3)                0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 10)                40        
                                                                 
Total params: 70
Trainable params: 70
Non-trainable params: 0
_________________________________________________________________


In [20]:
tf.keras.backend.clear_session()
target_model = build_target_model()
target_model.summary()

Model: "target_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 3)         30        
                                                                 
 global_average_pooling2d (G  (None, 3)                0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 10)                40        
                                                                 
Total params: 70
Trainable params: 70
Non-trainable params: 0
_________________________________________________________________


In [70]:
def siamese_network():
    inputs = layers.Input(shape=(28,28,1))
    # Init anchor model
    anchor_model = build_anchor_model()
    # Init target model without trainable params.
    target_model = build_target_model()
    target_model.trainable = False

    z1 = anchor_model(inputs)
    z2 = target_model(inputs)

    return models.Model(inputs, outputs=[z1, z2])

In [73]:
model = siamese_network()
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 anchor_model (Functional)      (None, 10)           70          ['input_11[0][0]']               
                                                                                                  
 target_model (Functional)      (None, 10)           70          ['input_11[0][0]']               
                                                                                                  
Total params: 140
Trainable params: 70
Non-trainable params: 70
__________________________________________________________________________________________________


In [72]:
model.compile(
    optimizer='adam',
    loss={
        'anchor_model': 'sparse_categorical_crossentropy',
        'target_model': 'sparse_categorical_crossentropy'
    }
)

model.fit(x, y, epochs=10)

Epoch 1/10
4/4 [==============================] - 1s 9ms/step - loss: 130.5201 - anchor_model_loss: 96.8079 - target_model_loss: 33.7122
Epoch 2/10
4/4 [==============================] - 0s 9ms/step - loss: 124.7629 - anchor_model_loss: 91.0507 - target_model_loss: 33.7122
Epoch 3/10
4/4 [==============================] - 0s 8ms/step - loss: 119.1330 - anchor_model_loss: 85.4208 - target_model_loss: 33.7122
Epoch 4/10
4/4 [==============================] - 0s 9ms/step - loss: 113.6204 - anchor_model_loss: 79.9082 - target_model_loss: 33.7122
Epoch 5/10
4/4 [==============================] - 0s 9ms/step - loss: 108.2178 - anchor_model_loss: 74.5056 - target_model_loss: 33.7122
Epoch 6/10
4/4 [==============================] - 0s 10ms/step - loss: 102.9128 - anchor_model_loss: 69.2006 - target_model_loss: 33.7122
Epoch 7/10
4/4 [==============================] - 0s 10ms/step - loss: 97.9094 - anchor_model_loss: 64.1971 - target_model_loss: 33.7122
Epoch 8/10
4/4 [========================

In [67]:
class EMA(tf.keras.callbacks.Callback):
    def __init__(self, decay=0.996):
        super(EMA, self).__init__()
        self.decay = decay

        # Create an ExponentialMovingAverage object
        self.ema = tf.train.ExponentialMovingAverage(decay=self.decay)

    def on_train_begin(self, logs=None):
        self.ema.apply(self.model.get_layer('anchor_model').trainable_variables)

    def on_epoch_end(self, epoch, logs=None):
        # Get exponential moving average of anchor model weights.
        train_vars = self.model.get_layer('anchor_model').trainable_variables
        averages = [self.ema.average(var) for var in train_vars]
        
        # Assign the average weights to target model
        target_model_vars = self.model.get_layer('target_model').non_trainable_variables
        assert len(target_model_vars) == len(averages)
        for i, var in enumerate(target_model_vars):
            var.assign(averages[i])

In [75]:
model.compile(
    optimizer='adam',
    loss={
        'anchor_model': 'sparse_categorical_crossentropy',
        'target_model': 'sparse_categorical_crossentropy'
    }
)

model.fit(x, y, epochs=10, callbacks=[EMA()])

Epoch 1/10
4/4 [==============================] - 1s 12ms/step - loss: 316.9131 - anchor_model_loss: 119.7637 - target_model_loss: 197.1494
Epoch 2/10
4/4 [==============================] - 0s 9ms/step - loss: 234.7221 - anchor_model_loss: 112.9389 - target_model_loss: 121.7832
Epoch 3/10
4/4 [==============================] - 0s 9ms/step - loss: 228.4099 - anchor_model_loss: 106.6267 - target_model_loss: 121.7832
Epoch 4/10
4/4 [==============================] - 0s 9ms/step - loss: 222.1068 - anchor_model_loss: 100.3236 - target_model_loss: 121.7832
Epoch 5/10
4/4 [==============================] - 0s 9ms/step - loss: 216.0723 - anchor_model_loss: 94.2892 - target_model_loss: 121.7832
Epoch 6/10
4/4 [==============================] - 0s 9ms/step - loss: 210.4549 - anchor_model_loss: 88.6717 - target_model_loss: 121.7832
Epoch 7/10
4/4 [==============================] - 0s 9ms/step - loss: 205.0095 - anchor_model_loss: 83.2263 - target_model_loss: 121.7832
Epoch 8/10
4/4 [=============